# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение информации об объекте, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [ ]:
import requests
import datetime as dt


def send_request(url, **params):
    with requests.get(url, params) as response:
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            return None


def get_apod_picture():
    while True:
        user_input = input("Введите дату в формате YYYY-MM-DD: ")
        if user_input.lower() == 'выйти':
            break
        try:
            date = dt.date.fromisoformat(user_input)
        except ValueError:
            print("Неверный ввод даты")
            continue
        else:
            url = "https://api.nasa.gov/planetary/apod"
            params = {
                "api_key": API_KEY,
                "date": date,
            }

            data = send_request(url, **params)
            if data == None:
                print("Что-то пошло не так...")
                break

            image_url = data["url"]
            explanation = data["explanation"]
            print(
                f"Ссылка на изображение: {image_url}\nОписание: {explanation}"
            )
            return {
                "url": image_url,
                "explanation": explanation,
            }

def get_mars_rover_photos():
    while True:
        user_input = input("Введите дату в формате YYYY-MM-DD: ")
        if user_input.lower() == 'выйти':
            break
        try:
            date = dt.date.fromisoformat(user_input)
        except ValueError:
            print("Неверный ввод даты")
            continue

        CAMERAS = ['FHAZ', 'RHAZ', 'MAST', 'CHEMCAM', 'MAHLI',
                   'MARDI', 'NAVCAM', 'PANCAM', 'MINITES']
        print('Список доступных камер: ', *CAMERAS, sep='\n\t')
        user_input = input(f"Выберите камеру из доступных: ")
        if user_input.lower() == 'выйти':
            break

        camera = user_input.upper()
        try:
            camera_index = CAMERAS.index(camera)
        except ValueError:
            print("Такой камеры не существует")
            continue
        else:
            url = "https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos"
            params = {
                "api_key": API_KEY,
                "earth_date": date,
                "camera": camera
            }

            data = send_request(url, **params)
            if data == None:
                print("Что-то пошло не так...")
                break

            photos = data["photos"]
            if not photos:
                return "Ничего не найдено"
            else:
                image_urls = list(photo["img_src"] for photo in photos)
                print('\n'.join(("Вот что удалось найти:", *image_urls)))
                return image_urls


def get_near_earth_objects_info():
    while True:
        user_input = input("Введите дату в формате YYYY-MM-DD: ")
        if user_input == "выйти":
            break
        try:
            date = dt.date.fromisoformat(user_input)
        except ValueError:
            print("Неверный ввод даты")
            continue
        else:
            url = "https://api.nasa.gov/neo/rest/v1/feed"
            params = {
                "api_key": API_KEY,
                "start_date": date,
                "end_date": date
            }

            data = send_request(url, **params)
            if data == None:
                print("Что-то пошло не так...")
                break

            objects_info = data["near_earth_objects"][date.isoformat()]
            objects_info = [
                {
                    "name": asteroid["name"],
                    "diameter": asteroid["estimated_diameter"]["meters"],
                    "is_potentially_hazardous_asteroid": asteroid[
                        "is_potentially_hazardous_asteroid"
                    ]
                }
                for asteroid in objects_info
            ]

            for asteroid in objects_info:
                for k, v in asteroid.items():
                    if type(v) == dict:
                        print(k, end=' - ')
                        for diameter_name, diameter_value in v.items():
                            print(diameter_name, diameter_value,
                                  sep=' = ', end = ' ')
                        print()
                    else:
                        print(k, v, sep=' - ')
                print()

            return objects_info


def get_space_weather():
    while True:
        user_input = input("Введите дату в формате YYYY-MM-DD: ")
        try:
            date = dt.date.fromisoformat(user_input)
        except:
            print("Неверный ввод даты")
            continue
        else:
            url = "https://api.nasa.gov/DONKI/notifications"
            params = {
                "startDate": date,
                "endDate": date,
                "api_key": API_KEY
            }

            user_input = input(
                "Какую информацию вы хотите получить\n" +
                "Магнитные бури / солнечные вспышки: "
            ).lower()
            if user_input == "магнитные бури":
                params["type"] = "gst"
            elif user_input == "солнечные вспышки":
                params["type"] = "flr"
            else:
                print("Неверный ввод")
                continue

            data = send_request(url, **params)
            if data == None:
                print("Что-то пошло не так...")
                break
            elif not data:
                print("Ничего не найдено")
                break

            notifications = []
            for notification in data:
                message = notification["messageBody"]
                message = message[message.find("## Summary:"):]
                notifications.append(message)

            print(f"{'_' * 80}\n".join(notifications))
            return notifications


def main():
    commands = ["выйти", "apod", "фото марсоходов", "приближающиеся объекты",
                "космическая погода"]
    print('список доступных комманд: ', *commands, sep='\n\t')
    while True:
        command = input().lower()

        if command == "выйти":
            break

        elif command == "apod":
            get_apod_picture()

        elif command == "фото марсоходов":
            get_mars_rover_photos()

        elif command == "приближающиеся объекты":
            get_near_earth_objects_info()

        elif command == "космическая погода":
            get_space_weather()

        else:
            print("Неизвестная команда")


API_KEY = input('Enter your api_key: ')

if __name__ == "__main__":
    main()

# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:

- Извлекать произведения искусства:
Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.

- Фильтровать произведения искусства:
Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).

- Отображать подробную информацию об оформлении:
Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.

- Разбивка на страницы и взаимодействие с пользователем:
Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

In [ ]:
import requests


def send_request(url, **params):
    global last_required_url

    with requests.get(url, params) as response:
        if response.ok:
            last_required_url = response.url
            data = response.json()
        else:
            response.raise_for_status()

    return data


def swap_page(page_number):
    url = last_required_url
    try:
        data = send_request(url, page=page_number)
    except requests.HTTPError as error:
        print(error)
    else:
        return data


def get_artworks(page=1):
    url = "https://api.artic.edu/api/v1/artworks"
    fields = ["id", "title"]
    params = {
        "page": page,
        "fields": ",".join(fields)
    }

    try:
        data = send_request(url, **params)
    except requests.HTTPError as error:
        print(error)
    else:
        return data


def filter_artworks(artist_name):
    url = "https://api.artic.edu/api/v1/artworks/search"
    fields = ["id", "title", "artist_ids", "artist_titles"]
    params = {
        "fields": ",".join(fields),
        "q": artist_name
    }

    try:
        data = send_request(url, **params)
    except requests.HTTPError as error:
        print(error)
    else:
        return data


def print_artworks(data):
    pagination = data["pagination"]
    current_page = pagination["current_page"]
    total_pages = pagination["total_pages"]

    print(f"Всего страниц: {total_pages}")
    print(f"Текущая страница: {current_page}")
    print("ID", "TITLE", sep="\t| ")
    for artwork in data["data"]:
        print(artwork["id"], artwork["title"], sep="\t| ")


def print_more_info_about_artwork():
    print("Чтобы получить подробную информацию об оформлении - введите его ID:")
    try:
        artwork_id = int(input())
    except ValueError:
        print("Неверный ввод")
    else:
        url = f"https://api.artic.edu/api/v1/artworks/{artwork_id}"
        fields = ["title", "artist_display",
                  "date_display", "artwork_type_title"]
        try:
            data = send_request(url, fields=",".join(fields))
        except requests.HTTPError as error:
            print(error)
        else:
            artwork_info = data["data"]

            artwork_title = artwork_info["title"]
            artwork_artists = artwork_info["artist_display"]
            artwork_date = artwork_info["date_display"]
            artwork_type = artwork_info["artwork_type_title"]

            artwork_info = {
                "Название произведения": artwork_title,
                "Исполнители": artwork_artists,
                "Дата" : artwork_date,
                "Носитель": artwork_type
            }
            for k, v in artwork_info.items():
                print(k, v.replace("\n", "\n\t"), sep=":\n\t")


def main():
    data = get_artworks()
    print_artworks(data)

    commands = {
        "Show all": "Вернуться на страницу со всеми произведениями",
        "Page": "выбрать страницу",
        "Next": "на следующую страницу",
        "Back": "На предыдущую страницу",
        "Artist": "получить произведения определённого исполнителя",
        "More": "Узнать подробнее о произведении",
        "Exit": "Выход из программы"
    }
    print("Список доступных команд:")
    for k, v in commands.items():
        print(k, v, sep=" - ")

    while True:
        pagination = data["pagination"]
        current_page = pagination["current_page"]
        total_pages = pagination["total_pages"]

        command = input("Введите команду: ").lower()

        if command == "page":
            try:
                page_number = int(input("Введите номер страницы: "))
            except ValueError:
                print("Неверный ввод")
            else:
                data = swap_page(page_number)
                print_artworks(data)

        elif command == "next":
            data = swap_page(current_page + 1)
            print_artworks(data)

        elif command == "back":
            data = swap_page(current_page - 1)
            print_artworks(data)

        elif command == "artist":
            artist_name = input("Введите имя исполнителя: ")
            data = filter_artworks(artist_name)
            print_artworks(data)

        elif command == "more":
            print_more_info_about_artwork()

        elif command == "exit":
            break

        else:
            print("Неизвестная команда")


last_required_url = ""

if __name__ == "__main__":
    main()

# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


In [ ]:
import requests


def send_request(url, **params):
    with requests.get(url, params) as response:
        if response.ok:
            data = response.json()
        else:
            response.raise_for_status()

    return data


def create_cryptocurrency_portfolio():
    pass


def main():
    pass


if __name__ == "__main__":
    main()

# Дополнительно: Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код